In [9]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QFileDialog, QVBoxLayout, QGraphicsView, QGraphicsScene, QTextBrowser, QScrollArea
from PyQt5.QtGui import QPen, QColor
from PyQt5.QtCore import Qt
import ezdxf

def parse_dxf_file(file_path):
    parsed_data = []
    doc = ezdxf.readfile(file_path)
    for entity in doc.modelspace():
        if entity.dxftype() == 'LINE':
            start_point = entity.dxf.start
            end_point = entity.dxf.end
            parsed_data.append({
                'type': 'LINE',
                'start_point': (start_point.x, start_point.y),
                'end_point': (end_point.x, end_point.y)
            })
        elif entity.dxftype() == 'CIRCLE':
            center = entity.dxf.center
            radius = entity.dxf.radius
            parsed_data.append({
                'type': 'CIRCLE',
                'center_point': (center.x, center.y),
                'radius': radius
            })
        elif entity.dxftype() == 'ARC':
            center = entity.dxf.center
            radius = entity.dxf.radius
            start_angle = entity.dxf.start_angle
            end_angle = entity.dxf.end_angle
            parsed_data.append({
                'type': 'ARC',
                'center_point': (center.x, center.y),
                'radius': radius,
                'start_angle': start_angle,
                'end_angle': end_angle
            })
        elif entity.dxftype() == 'LWPOLYLINE':
            polyline_data = []
            for vertex in entity.points():
                polyline_data.append((vertex.x, vertex.y))
            parsed_data.append({
                'type': 'POLYLINE',
                'vertices': polyline_data
            })

    return parsed_data, doc

def generate_gcode_from_dxf(parsed_data):
    gcode = []

    for entity in parsed_data:
        if entity['type'] == 'LINE':
            start_x, start_y = entity['start_point']
            end_x, end_y = entity['end_point']
            gcode.append(f'M01 X{end_x:.3f} Y{end_y:.3f}')
        elif entity['type'] == 'CIRCLE':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            gcode.append(f'M02 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f}')
            gcode.append(f'M03 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f}')
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']

            if start_angle < end_angle:
                gcode.append(f'M02 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
            else:
                gcode.append(f'M03 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
        elif entity['type'] == 'POLYLINE':
            vertices = entity['vertices']
            for vertex in vertices:
                x, y = vertex
                gcode.append(f'M01 X{x:.3f} Y{y:.3f}')

    return gcode

def calculate_drawing_extents(entities):
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')

    for entity in entities:
        if entity['type'] == 'LINE':
            start_x, start_y = entity['start_point']
            end_x, end_y = entity['end_point']
            min_x = min(min_x, start_x, end_x)
            max_x = max(max_x, start_x, end_x)
            min_y = min(min_y, start_y, end_y)
            max_y = max(max_y, start_y, end_y)
        elif entity['type'] == 'CIRCLE':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'POLYLINE':
            for vertex in entity['vertices']:
                x, y = vertex
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)

    return min_x, min_y, max_x, max_y

class DXFParserGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Display the DXF drawing
        self.view = QGraphicsView(self)
        self.scene = QGraphicsScene()
        self.view.setScene(self.scene)
        layout.addWidget(self.view)

        # Open DXF file button
        self.btn_open = QPushButton("Open DXF File")
        self.btn_open.clicked.connect(self.openFile)
        layout.addWidget(self.btn_open)

        # Scrollable G-code display
        self.gcode_browser = QTextBrowser(self)
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidget(self.gcode_browser)
        layout.addWidget(self.scroll_area)

        # Save G-code button
        self.btn_save = QPushButton("Save G-code")
        self.btn_save.clicked.connect(self.saveGCode)
        layout.addWidget(self.btn_save)

        self.setLayout(layout)
        self.setGeometry(100, 100, 1600, 1200)
        self.setWindowTitle("DXF Parser")
        self.show()

    def openFile(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Open DXF File", "", "DXF Files (*.dxf);;All Files (*)", options=options)
        if file_path:
            entities, self.doc = parse_dxf_file(file_path)
            self.parseAndDisplayDXF(entities)
            

    def parseAndDisplayDXF(self, entities):
        self.scene.clear()
        # Get the drawing extents for scaling
        min_x, min_y, max_x, max_y = calculate_drawing_extents(entities)
        drawing_width = max_x - min_x
        drawing_height = max_y - min_y
        scale_factor = 1000 / max(drawing_width, drawing_height)  # Adjust the scale as needed

        for entity in entities:
            if entity['type'] == 'LINE':
                start_x, start_y = entity['start_point']
                end_x, end_y = entity['end_point']
                line = self.scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                line.setPen(pen)
            elif entity['type'] == 'CIRCLE':
                center_x, center_y = entity['center_point']
                radius = entity['radius']
                circle = self.scene.addEllipse((center_x - radius) * scale_factor, (-center_y - radius) * scale_factor, radius * 2 * scale_factor, radius * 2 * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                circle.setPen(pen)
            elif entity['type'] == 'POLYLINE':
                vertices = entity['vertices']
                polyline = QGraphicsView()
                poly_scene = QGraphicsScene()
                polyline.setScene(poly_scene)
                pen = QPen(QColor(0, 0, 255))
                for i in range(len(vertices) - 1):
                    start_x, start_y = vertices[i]
                    end_x, end_y = vertices[i + 1]
                    line = poly_scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                    line.setPen(pen)
                self.view.setScene(self.scene)

    def saveGCode(self):
        gcode_text = self.gcode_browser.toPlainText()
       
def main():
    app = QApplication(sys.argv)
    window = DXFParserGUI()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0